# Abstract

Collecting the data from https://www.kaggle.com/c/word2vec-nlp-tutorial/data. The labeled data set consists of 25,000 IMDB movie reviews labeled with sentiment analysis, and 75,000 review test set without sentiment analysis. 
I am training a model to distinguish the review is postive or negative. 

I tried four models to get the better scores as possiable. For the model 1 and model 2, I'm using Word2vec to cleaning the data and convert it into vectors. Model 1 analyzes the review's sentiment by calculating the Average Feature Vectors to. Model 2 is using Cluster Analysis. And for the model 3, I am using the Bag-of-words model to process the data. 

Overall, Model 3 gets the best score, which is 0.8832


# Data Analysis

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk.data
from sklearn.cluster import KMeans
import time
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression
from random import shuffle
import heapq 
from utils.feature_select import select_feature
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from scipy.sparse import bsr_matrix


# download stopwords
import nltk
nltk.download('stopwords')

# Import the stop word list
from nltk.corpus import stopwords 

# word2vec
import logging
from gensim.models import word2vec

# forest
from sklearn.ensemble import RandomForestClassifier





[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bonnie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import Data

In [2]:
train = pd.read_csv("data/labeledTrainData.tsv", header=0, 
                     delimiter="\t", quoting=3)

test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv("data/unlabeledTrainData.tsv", header=0, 
                              delimiter="\t", quoting=3 )

# Makesure all the of reviews were read (100,000 in total)
print("Read %d labeled train reviews, %d test reviews, and %d unlabeled reviews\n" 
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 test reviews, and 50000 unlabeled reviews



In [3]:
print (train.head())
print(test.head())

         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...
           id                                             review
0  "12311_10"  "Naturally in a film who's main themes are of ...
1    "8348_2"  "This movie is a disaster within a disaster fi...
2    "5828_4"  "All in all, this is a movie for kids. We saw ...
3    "7186_2"  "Afraid of the Dark left me with the impressio...
4   "12128_7"  "A very accurate depiction of small time mob l...


In [4]:
## the first one review 
train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## Word2vec Model

In [5]:
# Function to convert data into wordlist
def review_to_wordlist(review, remove_stopwords=False):
    # Remove HTML
    review_text = BeautifulSoup(review, 'lxml').get_text()
      
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # Convert words to lower case and split them
    words = review_text.lower().split()

    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # Return a list of words
    return(words)

In [6]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
    return sentences

In [7]:
# the list of the first sentense
s1 = review_to_sentences(train['review'][0], tokenizer)
s1[0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again']

In [8]:
sentences = []
print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/bonnie/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/bonnie/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/bonnie/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/bonnie/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/bonnie/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client 

In [9]:
# totak amount of sentences 
len(sentences)

795538

In [10]:
print(sentences[0])

print(sentences[1])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [13]:
# import logging module to make output messages nicer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, 
            size=num_features, min_count = min_word_count, 
            window = context, sample = downsampling)

model.init_sims(replace=True)
model_name = 'model/300features_40minwords_10context'
model.save(model_name)

2019-04-03 19:44:01,954 : INFO : collecting all words and their counts
2019-04-03 19:44:01,956 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-03 19:44:02,012 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-04-03 19:44:02,075 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-04-03 19:44:02,117 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2019-04-03 19:44:02,178 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-04-03 19:44:02,227 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-04-03 19:44:02,283 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-04-03 19:44:02,350 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-04-03 19:44:02,405 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-04-03 19:44:02,456 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-04-03 19:44:02,504 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-04-03 19:44:02,552 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-04-03 19:44:02,600 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

2019-04-03 19:44:05,791 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-04-03 19:44:05,837 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-04-03 19:44:05,882 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-04-03 19:44:05,926 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-04-03 19:44:05,951 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-04-03 19:44:05,952 : INFO : Loading a fresh vocabulary
2019-04-03 19:44:06,023 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-04-03 19:44:06,023 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2019-04-03 19:44:06,072 : INFO : deleting the raw counts dictionary of 123504 items
2019-04-03 19:44:06,077 : I

2019-04-03 19:44:58,354 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-03 19:44:58,361 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-03 19:44:58,369 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-03 19:44:58,376 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-03 19:44:58,378 : INFO : EPOCH - 3 : training on 17798082 raw words (12749641 effective words) took 19.4s, 656951 effective words/s
2019-04-03 19:44:59,388 : INFO : EPOCH 4 - PROGRESS: at 6.90% examples, 878628 words/s, in_qsize 7, out_qsize 0
2019-04-03 19:45:00,389 : INFO : EPOCH 4 - PROGRESS: at 13.99% examples, 886397 words/s, in_qsize 7, out_qsize 0
2019-04-03 19:45:01,392 : INFO : EPOCH 4 - PROGRESS: at 21.29% examples, 897429 words/s, in_qsize 7, out_qsize 0
2019-04-03 19:45:02,396 : INFO : EPOCH 4 - PROGRESS: at 28.43% examples, 899163 words/s, in_qsize 7, out_qsize 0
2019-04-03 19:45:03,409 : INFO : EPOCH 4 - PR

## MODEL 1 : Average Feature Vectors

In [15]:
# Function to average all of the word vectors 
def makeFeatureVec(words, model, num_features):
    # initialize an empty numpy array 
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    
    # Index2word is a list contains the names of the words in the model's vocabulary. 
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
    
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# function to calculate the average feature vector for each one 
def getAvgFeatureVecs(reviews, model, num_features): 
    # Initialize a counter
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    for review in reviews:
       
        # Print a status message every 1000th review
        if counter%1000 == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
       
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

In [16]:
# Calculate average feature vectors for training and testing sets

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


/Users/bonnie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [18]:
# Fit a random forest to the training data, using 100 trees

forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

result = forest.predict( testDataVecs )
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( 'result/Word2_Ave.csv', index=False, quoting=3 )

Fitting a random forest to labeled training data...


## MODEL 1 : Conclusion


![title](pic/Word2_Ave.png)

Word2vec is the most popular way of processing the words to anaylize, that's why i am using that for the first two of my models. Analyze the data by using Average Feature Vectors, which gets socres of 0.82748.

## MODEL 2 : Cluster Analysis

In [21]:
# Start time
start = time.time() 

# Set "k" (num_clusters) to get an average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] // 5

kmeans_clustering = KMeans( n_clusters = num_clusters, n_jobs=-2 )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

/Users/bonnie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


Time taken for K Means clustering:  402.3447847366333 seconds.


In [22]:
# Mapping each vocabulary word to a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [23]:
# For the first 10 clusters
for cluster in range(0, 10):

    print("\nCluster %d" % cluster)

    # print all of the words for that cluster number
    words = []
    for i in range(0, len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['contrasting']

Cluster 1
['fury', 'leprechaun', 'wolfman', 'knockoff']

Cluster 2
['esther', 'pammy']

Cluster 3
['sneaking']

Cluster 4
['kill', 'survive', 'destroy', 'defeat', 'defend', 'conquer', 'eliminate', 'invent', 'dispose', 'summon', 'unleash', 'consume', 'thwart']

Cluster 5
['jan', 'meyers', 'moriarty', 'pare', 'wilding', 'channeling', 'jayston']

Cluster 6
['agents', 'investigators']

Cluster 7
['helmet', 'fried', 'straw', 'torch', 'frog', 'bowling', 'horns', 'slime', 'flaming', 'grinning', 'fangs', 'statues', 'wasp', 'vinyl', 'tattoos', 'goblin', 'feces']

Cluster 8
['overt', 'metaphysical', 'freudian']

Cluster 9
['replaced', 'confronted', 'approached', 'greeted']


In [24]:
def create_bag_of_centroids( wordlist, word_centroid_map ):

    # The number of clusters is equal to the highest cluster index
    num_centroids = max( word_centroid_map.values() ) + 1
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )

    # find which cluster it belongs to
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1

    return bag_of_centroids

In [25]:
train_centroids = np.zeros((train["review"].size, num_clusters), dtype="float32")

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

test_centroids = np.zeros(( test["review"].size, num_clusters), dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map )
    counter += 1

In [26]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "result/BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


## MODEL 2 : Conclusion

![title](pic/BOC.png)


Model 2 is based on word2vec model to analze the data by using Cluster, which gets the socre as 0.84776.

## MODEL 3 : Bag-of-words model

In [4]:
# count all the vector & select the feature 
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=None,
                             max_features=5000)
 
corpus_simple = ['I like you yo, do you like me?', 'yo yo hello']
train_data_features = vectorizer.fit_transform(corpus_simple)
 
print('feature value: ', train_data_features.toarray())
print('feature name: ', vectorizer.get_feature_names())
 
print('vectorizer.vocabulary_: ', vectorizer.vocabulary_)
 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
print('feature count: ', len(vectorizer.vocabulary_))

feature value:  [[1 0 2 1 1 2]
 [0 1 0 0 2 0]]
feature name:  ['do', 'hello', 'like', 'me', 'yo', 'you']
vectorizer.vocabulary_:  {'like': 2, 'you': 5, 'yo': 4, 'do': 0, 'me': 3, 'hello': 1}
feature count:  5000


In [5]:
# function to process the word using chisquare
def chi(term, cl, X, Y):
    N = len(Y)
    A = B = C = D = 0
    for i, x in enumerate(X):
        flag = False
        #for k in x.keys():
        for k in x:
            if k == term:
                flag = True
                break
 
        if flag == True:
            if str(Y[i]) == str(cl):
                A = A + 1
            else:
                B = B + 1
        else:
            if str(Y[i]) == str(cl):
                C = C + 1
            else:
                D = D + 1
    re = int((N * (A * D - B * C) * (A * D - B * C)) / ((A + B) * (A + C) * (B + D) * (C + D)))
    return re

## Traversing the words in bags
def dump_feature(labels, dic, X, Y, filePath):

    fp = open(filePath, 'w')
    dic_num = len(dic)
 

    for cl in labels:
        fp.write(str(cl))
        i = 0
        for term in dic.keys():
            if ((i+1)%1000 == 0):
                print("term %d of %d" % (i+1, dic_num))
            s = chi(term, cl, X, Y)
            fp.write(' ' + term + ':' + str(s))
            i+=1
        fp.write('\n')
        fp.flush()
    fp.close()
    
labels = [0, 1]
file_path = 'result/feature_word.txt'
X = [review.split(' ') for review in clean_train_reviews]
 
print('Start...')
dump_feature(labels, vectorizer.vocabulary_, X, train['sentiment'], filePath=file_path)

Start...
term 1000 of 5000
term 2000 of 5000
term 3000 of 5000
term 4000 of 5000
term 5000 of 5000
term 1000 of 5000
term 2000 of 5000
term 3000 of 5000
term 4000 of 5000
term 5000 of 5000


In [6]:
## select the most k features
def select_feature(filePath, k):
    read = open(filePath, 'r')
    lab_fea = {}
 
    for line in read:
        line_arr = line.strip().split()
        if len(line_arr) - 1 <= k:
            lab_fea[line_arr[0]] = [kv.split(':')[0] for kv in line_arr[1:]]
        else:
            heap = []
            heapq.heapify(heap)
            for kv in line_arr[1:]:
                key, val = kv.split(':')
                if len(heap) < k:
                    heapq.heappush(heap, (float(val), key))
                else:
                    if float(val) > heap[0][0]:
                        heapq.heappop(heap)
                        heapq.heappush(heap, (float(val), key))
            lab_fea[line_arr[0]] = [heapq.heappop(heap)[1] for i in range(len(heap))]
    read.close()
    return lab_fea
 
## select the top 1000
max_feature = 1000
fab_feature = select_feature('result/feature_word.txt', max_feature)["1"]
 
# put in tf-idf model
vectorizer = TfidfVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 stop_words = None,
                                 vocabulary = fab_feature,
                                 max_features = max_feature)

In [8]:
vectorizer = TfidfVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 stop_words = None,
                                 vocabulary = None, # fab_feature
                                 max_features = None)
 
corpus_simple = ['I like you yo, do you like me?', 'yo yo hello']
 
train_data_features_simple = vectorizer.fit_transform(corpus_simple)

print('feature value: ', train_data_features_simple.toarray())
print('feature value shape: ', train_data_features_simple.shape)
print('feature name: ', vectorizer.get_feature_names())
 
train_data_features_simple = bsr_matrix(train_data_features_simple)
print('bsr_matrix_train_data_features_simple: \n', train_data_features_simple)
print('bsr_matrix_train_data_features_simple.shape: ', train_data_features_simple.shape)

feature value:  [[0.30851498 0.         0.61702996 0.30851498 0.21951095 0.61702996]
 [0.         0.57496187 0.         0.         0.81818021 0.        ]]
feature value shape:  (2, 6)
feature name:  ['do', 'hello', 'like', 'me', 'yo', 'you']
bsr_matrix_train_data_features_simple: 
   (0, 2)	0.6170299644182933
  (0, 5)	0.6170299644182933
  (0, 4)	0.21951095080860833
  (0, 0)	0.30851498220914664
  (0, 3)	0.30851498220914664
  (1, 4)	0.8181802073667197
  (1, 1)	0.5749618667993135
bsr_matrix_train_data_features_simple.shape:  (2, 6)


In [9]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)
print('train_data_features.shape: ', train_data_features.shape)
 
train_data_features = bsr_matrix(train_data_features)
print('bsr_matrix_train_data_features.shape: ', train_data_features.shape)
 
# put in LR to train
lr = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1, fit_intercept=True, intercept_scaling=1.0, class_weight=None, random_state=None) 
lr.fit(train_data_features, list(train["sentiment"]))

train_data_features.shape:  (25000, 74047)
bsr_matrix_train_data_features.shape:  (25000, 74047)


/Users/bonnie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1.0, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
test = pd.read_csv("data/testData.tsv", header = 0, delimiter = "\t", quoting = 3)
 
clean_test_reviews = []
for i in range(len(test)):
    clean_review = review_to_words(test['review'][i])
    clean_test_reviews.append(clean_review)
   
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = bsr_matrix(test_data_features)
 
result = lr.predict(test_data_features)

In [11]:
df = pd.DataFrame(data={"id": test["id"], "sentiment": result})
df.to_csv("result/bow__tfidf_lr.csv", index=False, quoting=3)

## MODEL 3 : Conclusion

![title](pic/chi.png)

Since the two models based on Word2vec model didn't get a good score. That's why, I am using another model to process the data, which is Bag-of-words model. This model taks the frequency of occurrence of words as a feature of the training classifier. And this model gets the highest score as 0.8832

# Conclusion

Analyzing the same dataset by using different methods can get different accuracy. The mainly different between the methods is how they processing the data. Like the first two models are using word2vec model, which put words into vector. But for the model that model 3 was using is Bag-of-words model, which is consider the frequency of occurrence of words as a feature to process. And from the way I am analyzing, the model 3 gets the highest score as 0.8832. Even though model 3 gets the highest accuracy, different combination of the method of processing data and the way to build the model might get better scores. 

- The different between mine work and the public kernels is that I am not just using one way to process the data. By using different methods, I get an different way to analyzing the data, which allow me to get a better scores.

# Contributions Statement
- Hsiang-Hua Chen : 30% 
- Internet : 70%

# Citations
- https://www.kaggle.com/c/word2vec-nlp-tutorial
- https://www.kaggle.com/c/word2vec-nlp-tutorial/submissions?sortBy=date&group=successful&page=1&pageSize=20
- https://en.wikipedia.org/wiki/Word2vec
- https://en.wikipedia.org/wiki/Bag-of-words_model
- https://www.kaggle.com/nilanml/imdb-review-deep-model-94-89-accuracy

# License

MIT License

Copyright (c) 2019 Hsiang-Hua Chen

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.